In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from zipfile import ZipFile
filename = "/content/drive/My Drive/HAM10000.zip"

with ZipFile(filename, 'r') as zip:
    zip.extractall()
    print("Dataset extracted")


In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from glob import glob
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation, MaxPooling2D, concatenate
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# Paths
train_df = pd.read_csv('/code/MyCode/AUG/HAM10000/train.csv')
test_df = pd.read_csv('/code/MyCode/AUG/HAM10000/test.csv')
train_path = '/code/MyCode/AUG/HAM10000/train_dir'
test_path = '/code/MyCode/AUG/HAM10000/test_dir'

targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
batch_size = 16
image_size = 224


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = datagen.flow_from_directory(
    directory=train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=True
)

test_batches = datagen.flow_from_directory(
    directory=test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False
)


In [ ]:
from keras import backend as K
from keras.layers import Layer
import keras.layers as kl

class SoftAttention(Layer):
    def __init__(self, ch, m, concat_with_x=False, aggregate=False, **kwargs):
        self.channels = int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x
        super(SoftAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.i_shape = input_shape
        kernel_shape_conv3d = (self.channels, 3, 3, 1, self.multiheads)
        self.out_attention_maps_shape = input_shape[0:1] + (self.multiheads,) + input_shape[1:-1]
        if self.aggregate_channels == False:
            self.out_features_shape = input_shape[:-1] + (input_shape[-1] + (input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1] + (input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                            initializer='he_uniform',
                                            name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                          initializer='zeros',
                                          name='bias_conv3d')
        super(SoftAttention, self).build(input_shape)

    def call(self, x):
        exp_x = K.expand_dims(x, axis=-1)
        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d, self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)
        conv3d = K.permute_dimensions(conv3d, pattern=(0,4,1,2,3))
        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d, shape=(-1, self.multiheads, self.i_shape[1]*self.i_shape[2]))
        softmax_alpha = K.softmax(conv3d, axis=-1)
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1], self.i_shape[2]))(softmax_alpha)

        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha, pattern=(0,2,3,1,4))
            x_exp = K.expand_dims(x, axis=-2)
            u = kl.Multiply()([exp_softmax_alpha, x_exp])
            u = kl.Reshape(target_shape=(self.i_shape[1], self.i_shape[2], u.shape[-1]*u.shape[-2]))(u)
        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha, pattern=(0,2,3,1))
            exp_softmax_alpha = K.sum(exp_softmax_alpha, axis=-1)
            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)
            u = kl.Multiply()([exp_softmax_alpha, x])

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u, x])
        else:
            o = u

        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape):
        return [self.out_features_shape, self.out_attention_maps_shape]


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# Load EfficientNetB0
efficientnet = EfficientNetB0(
    include_top=True,
    weights='imagenet',
    input_shape=(image_size, image_size, 3)
)

# Exclude last 28 layers for attention
conv = efficientnet.layers[-28].output

# Soft Attention
attention_layer, map2 = SoftAttention(
    aggregate=True, m=16, concat_with_x=False, ch=int(conv.shape[-1]), name='soft_attention'
)(conv)

attention_layer = MaxPooling2D(pool_size=(2,2), padding="same")(attention_layer)
conv = MaxPooling2D(pool_size=(2,2), padding="same")(conv)

conv = concatenate([conv, attention_layer])
conv = Activation('relu')(conv)
conv = Dropout(0.5)(conv)

output = Flatten()(conv)
output = Dense(7, activation='softmax')(output)

# Model
model = Model(inputs=efficientnet.input, outputs=output)

# Compile
opt1 = tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=0.1)
model.compile(optimizer=opt1, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
checkpoint = ModelCheckpoint(
    filepath='/content/drive/My Drive/EfficientNetB0+SA.hdf5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True
)
earlystop = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=40,
    min_delta=0.001
)

history = model.fit(
    train_batches,
    steps_per_epoch=len(train_df)//batch_size,
    epochs=50,
    verbose=2,
    validation_data=test_batches,
    validation_steps=len(test_df)//batch_size,
    callbacks=[checkpoint, earlystop]
)


In [ ]:
# Load best weights
model.load_weights('/content/drive/My Drive/EfficientNetB0+SA.hdf5')

predictions = model.predict(test_batches, steps=len(test_df)//batch_size, verbose=0)
y_pred = np.argmax(predictions, axis=1)
y_true = test_batches.classes
y_prob = predictions
y_test = to_categorical(y_true)

# Classification report
print(classification_report(y_true, y_pred, target_names=targetnames))

# Confusion matrix
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, targetnames)


In [ ]:
fpr, tpr, roc_auc = {}, {}, {}
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(f"ROC AUC {targetnames[i]}: {roc_auc[i]:.3f}")

plt.figure(figsize=(8,6))
colors = ['v-', 'c', 'b', 'g', 'y', 'o-', 'r']
for i, color in enumerate(colors):
    plt.plot(fpr[i], tpr[i], color, label=f'{targetnames[i]} (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves EfficientNetB0+SA')
plt.legend(loc="lower right")
plt.show()


In [ ]:
sa_model = Model(model.inputs, model.get_layer('soft_attention').output)

for i in range(len(test_batches)):
    x_test, y_test_batch = test_batches[i]
    sa_features, sa_maps = sa_model.predict(x_test)
    class_predictions = model.predict(x_test)

    for j in range(len(x_test)):
        t = (x_test + 1)/2
        fig, axes = plt.subplots(2,1, figsize=(5,10))
        axes[0].imshow(t[j])
        pred_idx = np.argmax(class_predictions[j])
        gt_idx = np.argmax(y_test_batch[j])
        pred = targetnames[pred_idx]
        gt = targetnames[gt_idx]
        sum_attnmap = np.sum(sa_maps[j], 0)
        axes[1].imshow(t[j], alpha=1.0)
        axes[1].imshow(cv2.resize(sum_attnmap, (224,224), interpolation=cv2.INTER_CUBIC), cmap='jet', alpha=0.5)
        axes[1].set_xlabel(f"Pred: {pred}, GT: {gt}")
        plt.tight_layout()
        if pred == gt:
            plt.savefig(f'densenet/trueCases/plt_{i}_{j}.jpg')
        else:
            plt.savefig(f'densenet/failCases/plt_{i}_{j}.jpg')
        plt.close()
